In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model, preprocessing, metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
import time

In [2]:
df = pd.read_csv('df_for_model.csv')
df = df.drop('Unnamed: 0',axis=1)

In [3]:
SiteEnergyUse_discrete = pd.qcut(df['SiteEnergyUse(kBtu)'],3, labels = False)
GHGEmission_discrete = pd.qcut(df['GHGEmissions(MetricTonsCO2e)'],3, labels = False)
df.insert(11, 'SiteEnergyUse_discrete', SiteEnergyUse_discrete)
df.insert(12, 'GHGEmission_discrete', GHGEmission_discrete)

In [4]:
df['SiteEnergyUse(kBtu)_log'] = np.log1p(df['SiteEnergyUse(kBtu)'])
df['GHGEmissions(MetricTonsCO2e)_log'] = np.log1p(df['GHGEmissions(MetricTonsCO2e)'])

In [5]:
df.columns

Index(['PrimaryPropertyType', 'Neighborhood', 'YearBuilt', 'NumberofBuildings',
       'NumberofFloors', 'PropertyGFAParking', 'PropertyGFABuilding(s)',
       'ListOfAllPropertyUseTypes', 'LargestPropertyUseType',
       'LargestPropertyUseTypeGFA', 'SecondLargestPropertyUseType',
       'SiteEnergyUse_discrete', 'GHGEmission_discrete',
       'SecondLargestPropertyUseTypeGFA', 'ENERGYSTARScore',
       'SiteEnergyUse(kBtu)', 'GHGEmissions(MetricTonsCO2e)',
       'MedianEnergyUse', 'MedianGHGEmission', 'NumberofUseTypes',
       'Laboratory', 'Wholesale Club/Supercenter', 'Residential Care Facility',
       'Refrigerated Warehouse', 'Other/Specialty Hospital',
       'Other - Technology/Science', 'Other - Mall', 'Convention Center',
       'Strip Mall', 'Vocational School', 'Police Station', 'Hotel',
       'Food Service', 'SPS-District K-12', 'Supermarket/Grocery Store',
       'Data Center', 'Urgent Care/Clinic/Other Outpatient', 'Library',
       'Automobile Dealership', 'Senior C

In [6]:
#df=df.drop(df[df['PrimaryPropertyType']=='Hospital'].index)

In [7]:
data=df['Neighborhood'].values.reshape(-1,1)
enc = preprocessing.OneHotEncoder()
neighborhood_enc = enc.fit_transform(data).toarray()
neighborhood_enc.shape

(1811, 13)

In [8]:
data=df['PrimaryPropertyType'].values.reshape(-1,1)
enc = preprocessing.OneHotEncoder()
PropertyType_enc = enc.fit_transform(data).toarray()
PropertyType_enc.shape

(1811, 25)

In [9]:
X_cat = np.concatenate((neighborhood_enc,PropertyType_enc),axis=1)
X_cat.shape

(1811, 38)

In [10]:
# X_cat = np.concatenate((neighborhood_enc,df[['Laboratory', 'Wholesale Club/Supercenter', 'Residential Care Facility',
#        'Refrigerated Warehouse', 'Other/Specialty Hospital',
#        'Other - Technology/Science', 'Other - Mall', 'Convention Center',
#        'Strip Mall', 'Vocational School', 'Police Station', 'Hotel',
#        'Food Service', 'SPS-District K-12', 'Supermarket/Grocery Store',
#        'Data Center', 'Urgent Care/Clinic/Other Outpatient', 'Library',
#        'Automobile Dealership', 'Senior Care Community',
#        'Other - Restaurant/Bar', 'Convenience Store without Gas Station',
#        'Parking', 'Hospital (General Medical & Surgical)',
#        'Prison/Incarceration', 'Worship Facility', 'Medical Office', 'Other',
#        'Bar/Nightclub', 'Other - Education', 'Self-Storage Facility',
#        'Other - Public Services', 'Movie Theater', 'Swimming Pool',
#        'Repair Services (Vehicle; Shoe; Locksmith; etc)', 'Single Family Home',
#        'Personal Services (Health/Beauty; Dry Cleaning; etc)',
#        'Energy/Power Station', 'Residence Hall/Dormitory',
#        'Other - Recreation', 'Social/Meeting Hall',
#        'Small- and Mid-Sized Office', 'Courthouse', 'Distribution Center',
#        'Other - Services', 'Museum', 'Retail Store', 'Office',
#        'College/University', 'Hospital', 'Fitness Center/Health Club/Gym',
#        'Fast Food Restaurant', 'Multifamily Housing', 'Lifestyle Center',
#        'Fire Station', 'Manufacturing/Industrial Plant', 'K-12 School',
#        'Other - Entertainment/Public Assembly', 'Bank Branch', 'Food Sales',
#        'Adult Education', 'Large Office', 'Non-Refrigerated Warehouse',
#        'Outpatient Rehabilitation/Physical Therapy', 'Mixed Use Property',
#        'Restaurant', 'Other - Lodging/Residential', 'Pre-school/Daycare',
#        'Enclosed Mall', 'Performing Arts', 'Other - Utility',
#        'Financial Office']].to_numpy()),axis=1)

In [11]:
X_quant = df[['YearBuilt', 'NumberofBuildings', 'NumberofFloors', 'PropertyGFABuilding(s)','PropertyGFAParking','LargestPropertyUseTypeGFA','SecondLargestPropertyUseTypeGFA','MedianEnergyUse','MedianGHGEmission','NumberofUseTypes']]
X_quant.values.shape

(1811, 10)

In [12]:
X_cat.shape

(1811, 38)

In [13]:
Y_energy_log = df['SiteEnergyUse(kBtu)_log']
Y_ghg_log = df['GHGEmissions(MetricTonsCO2e)_log']
Y_energy_bis = df['SiteEnergyUse(kBtu)']
Y_ghg_bis = df['GHGEmissions(MetricTonsCO2e)']

In [14]:
X_cat_train, X_cat_test, X_quant_train, X_quant_test, Y_energy_log_train, Y_energy_log_test, Y_ghg_log_train, Y_ghg_log_test, Y_energy_bis_train, Y_energy_bis_test, Y_ghg_bis_train, Y_ghg_bis_test = train_test_split(X_cat, X_quant, Y_energy_log, Y_ghg_log, Y_energy_bis, Y_ghg_bis, test_size=0.3, stratify = df['PrimaryPropertyType'], random_state = 1)

In [15]:
std = preprocessing.StandardScaler()
X_quant_train_scaled = std.fit_transform(X_quant_train)
X_quant_test_scaled = std.transform(X_quant_test)

In [16]:
X_train = np.concatenate((X_cat_train,X_quant_train_scaled),axis=1)
X_test = np.concatenate((X_cat_test,X_quant_test_scaled),axis=1)

In [145]:
# Régression linéaire sur cibles en log
lr_energy = linear_model.LinearRegression()
lr_energy.fit(X_train,Y_energy_log_train)

print('R² test sur variable log:', round(metrics.r2_score(Y_energy_log_test,lr_energy.predict(X_test)),3))
print('R² test sur variable origine:', round(metrics.r2_score(np.expm1(Y_energy_log_test),np.expm1(lr_energy.predict(X_test))),3))

print('R² train sur variable log:',round(metrics.r2_score(Y_energy_log_train,lr_energy.predict(X_train)),3))
print('R² train sur variable orig:',round(metrics.r2_score(np.expm1(Y_energy_log_train),np.expm1(lr_energy.predict(X_train))),3))

R² test sur variable log: -0.19
R² test sur variable origine: -4.850533215057446e+22
R² train sur variable log: 0.579
R² train sur variable orig: -12.136


In [146]:
# Régression linéaire sur cibles non transformées
lr_energy_bis = linear_model.LinearRegression()
lr_energy_bis.fit(X_train,Y_energy_bis_train)

print('R² test :', round(metrics.r2_score(Y_energy_bis_test,lr_energy_bis.predict(X_test)),3))
print('R² train :',round(metrics.r2_score(Y_energy_bis_train,lr_energy_bis.predict(X_train)),3))

R² test : 0.89
R² train : 0.593


In [19]:
recap_energy = pd.DataFrame(columns = ['R²','RMSE','Prediction time','Best parameters']).rename_axis('Model for energy')
recap_ghg = pd.DataFrame(columns = ['R²','RMSE','Prediction time','Best parameters']).rename_axis('Model for GHG')

In [20]:
recap_energy_log = pd.DataFrame(columns = ['R²','RMSE','Prediction time','Best parameters']).rename_axis('Model for energy log')
recap_ghg_log = pd.DataFrame(columns = ['R²','RMSE','Prediction time','Best parameters']).rename_axis('Model for GHG log')

In [21]:
def perf(X_train,Y_train,X_test,Y_test,model,hyperparam):
    
    '''Fonction permettant la recherche des meilleurs hyperparamètres d'un modèle puis 
    l'entrainement de ce modèle avec ces meilleurs paramètres sur le jeu X_train, Y_train
    La fonction retourne les performances du modèle (R², RMSE, temps d'éxécution) et les meilleurs paramètres
    
    le paramètre 'hyperparam' de la fonction est un dictionnaire
    
    Liste des hyperparamètres selon certains modèles :
    - Régression linéaire : pas d'hyperparamètre
    - Régression Ridge : alpha
    - Regression Lasso : alpha
    - Elasticnet : alpha et l1_ratio
    - SVR : estimator__C, estimator__epsilon et estimator__kernel
    - Random Forest : 
    '''
        
    model_gs = GridSearchCV(estimator = model, param_grid=hyperparam, cv=5)      
        
    model_gs.fit(X_train, Y_train)
    start_time = time.time()
    score = round(metrics.r2_score(Y_test,model_gs.predict(X_test)),5)     
    RMSE = round(metrics.mean_squared_error(Y_test,model_gs.predict(X_test), squared = False),5)
    pred_time = round(time.time()-start_time, 6)
    best_param = model_gs.best_params_
    for k, v in best_param.items():
        if type(v)!= str :
            best_param[k] = round(v, 6)
   
    return score, RMSE, pred_time, best_param



def recap_table(X_train,Y_energy_train,Y_ghg_train,X_test,Y_energy_test,Y_ghg_test,model,hyperparam):
    score, RMSE, pred_time, best_param = perf(X_train,Y_energy_train,X_test,Y_energy_test,model,hyperparam)
    recap_energy.loc[str(model),:] = [score,RMSE,pred_time,best_param]
    
    score, RMSE, pred_time, best_param = perf(X_train,Y_ghg_train,X_test,Y_ghg_test,model,hyperparam)
    recap_ghg.loc[str(model),:] = [score,RMSE,pred_time,best_param]

In [22]:
def perf_log(X_train,Y_train,X_test,Y_test,model,hyperparam):
    
    '''Fonction permettant la recherche des meilleurs hyperparamètres d'un modèle puis 
    l'entrainement de ce modèle avec ces meilleurs paramètres sur le jeu X_train, Y_train
    La fonction retourne les performances du modèle (R², RMSE, temps d'éxécution) et les meilleurs paramètres
    
    le paramètre 'hyperparam' de la fonction est un dictionnaire
    
    Liste des hyperparamètres selon certains modèles :
    - Régression linéaire : pas d'hyperparamètre
    - Régression Ridge : alpha
    - Regression Lasso : alpha
    - Elasticnet : alpha et l1_ratio
    - SVR : estimator__C, estimator__epsilon et estimator__kernel
    - Random Forest : 
    '''
        
    model_gs = GridSearchCV(estimator = model, param_grid=hyperparam, cv=5)      
        
    model_gs.fit(X_train, Y_train)
    start_time = time.time()
    score = round(metrics.r2_score(np.expm1(Y_test),np.expm1(model_gs.predict(X_test))),5)     
    RMSE = round(metrics.mean_squared_error(np.expm1(Y_test),np.expm1(model_gs.predict(X_test)), squared = False),5)
    pred_time = round(time.time()-start_time, 6)
    best_param = model_gs.best_params_
    for k, v in best_param.items():
        if type(v)!= str :
            best_param[k] = round(v, 6)
   
    return score,RMSE, pred_time, best_param



def recap_table_log(X_train,Y_energy_train,Y_ghg_train,X_test,Y_energy_test,Y_ghg_test,model,hyperparam):
    score, RMSE, pred_time, best_param = perf_log(X_train,Y_energy_train,X_test,Y_energy_test,model,hyperparam)
    recap_energy_log.loc[str(model),:] = [score,RMSE,pred_time,best_param]
    
    score, RMSE, pred_time, best_param = perf_log(X_train,Y_ghg_train,X_test,Y_ghg_test,model,hyperparam)
    recap_ghg_log.loc[str(model),:] = [score,RMSE,pred_time,best_param]

In [23]:
LinearReg = linear_model.LinearRegression();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,LinearReg,{})

In [24]:
ridge = Ridge();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,ridge,{'alpha': np.logspace(-4,2,100)})

In [25]:
lasso = Lasso();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,lasso,{'alpha': np.logspace(-4,2,100)})

C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 8.307471384558938e+16, tolerance: 56314712393550.78
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.0409124607389932e+16, tolerance: 35302160439502.13
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9.724723244846925e+16, tolerance: 63166560028773.97
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:5

In [26]:
svr = SVR(kernel = 'linear');
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,svr,{'C' : range(1,5),
                                       'epsilon' : np.linspace(0.1,1,10)})

In [27]:
svr = SVR();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,svr,{'C' : range(1,5),
                                        'epsilon' : np.linspace(0.1,1,10), 
                                       'kernel' : ['rbf','poly','sigmoid']})

In [28]:
rfr = RandomForestRegressor();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test, rfr, {'min_samples_split' : [2, 5, 10], 
                                             'min_samples_leaf' : [1, 2, 4], 
                                             'max_depth': range(5,16,1)})

In [29]:
gb = LGBMRegressor();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test, gb, {'n_estimators': [150],'max_depth':range(2,16), 
                                                                                                     'learning_rate' : [0.0001,0.001,0.01,0.1], 
                                                                                                     'boosting_type' : ['gbdt','goss', 'dart']})

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^m

In [30]:
recap_energy

,R²,RMSE,Prediction time,Best parameters
Model for energy,,,,
LinearRegression(),0.88964,1.35654e+07,0.001999,{}
Ridge(),0.85202,1.57079e+07,0.003003,{'alpha': 100.0}
Lasso(),0.89091,1.34871e+07,0.002,{'alpha': 100.0}
SVR(kernel='linear'),-0.02273,4.12955e+07,0.073006,"{'C': 4, 'epsilon': 0.1}"
SVR(),0.22355,3.59815e+07,0.092005,"{'C': 4, 'epsilon': 0.1, 'kernel': 'poly'}"
RandomForestRegressor(),0.3066,3.40027e+07,0.047004,"{'max_depth': 13, 'min_samples_leaf': 4, 'min_..."
LGBMRegressor(),0.23544,3.57048e+07,0.008,"{'boosting_type': 'goss', 'learning_rate': 0.0..."


In [42]:
recap_ghg

,R²,RMSE,Prediction time,Best parameters
Model for GHG,,,,
LinearRegression(),0.5965,485.928,0.002007,{}
Ridge(),0.73413,394.443,0.002002,{'alpha': 100.0}
Lasso(),0.76788,368.557,0.002002,{'alpha': 43.287613}
SVR(kernel='linear'),0.62345,469.418,0.068004,"{'C': 3, 'epsilon': 1.0}"
SVR(),0.07026,737.614,0.120003,"{'C': 4, 'epsilon': 1.0, 'kernel': 'sigmoid'}"
RandomForestRegressor(),0.57454,498.974,0.044006,"{'max_depth': 7, 'min_samples_leaf': 4, 'min_s..."
LGBMRegressor(),0.70083,418.414,0.012003,"{'boosting_type': 'goss', 'learning_rate': 0.1..."


In [32]:
LinearReg = linear_model.LinearRegression();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,LinearReg,{})

In [33]:
ridge = Ridge();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,ridge,{'alpha': np.logspace(-4,2,100)})

In [34]:
lasso = Lasso();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,lasso,{'alpha': np.logspace(-4,2,100)})

C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 104.02700961171104, tolerance: 0.18940016219726957
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 251.3970099542744, tolerance: 0.18948116299514178
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 242.72261475019727, tolerance: 0.19359686915879712
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: C

In [35]:
svr = SVR(kernel = 'linear');
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,svr,{'C' : range(1,5),
                                       'epsilon' : np.linspace(0.1,1,10)})

In [36]:
svr = SVR();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,svr,{'C' : range(1,5),
                                        'epsilon' : np.linspace(0.1,1,10), 
                                       'kernel' : ['rbf','poly','sigmoid']})

In [143]:
rfr = RandomForestRegressor();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test, rfr, {'n_estimators':[50],'min_samples_split' : [2], 
                                             'min_samples_leaf' : [2], 'random_state':[0],
                                             'max_depth': [11]})

In [38]:
gb = LGBMRegressor();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test, gb, {'n_estimators': range(300,500,10),'max_depth':[2,3,4], 
                                                                                                     'learning_rate' : [0.0001, 0.001,0.01,0.1], 
                                                                                                     'boosting_type' : ['gbdt','goss', 'dart']})

In [39]:
recap_energy_log

,R²,RMSE,Prediction time,Best parameters
Model for energy log,,,,
LinearRegression(),-4.85053e+22,8.99325e+18,0.002985,{}
Ridge(),-3.8982e+23,2.5495e+19,0.002,{'alpha': 2.31013}
Lasso(),-7.53376e+23,3.54428e+19,0.003007,{'alpha': 0.002154}
SVR(kernel='linear'),-8.92957e+29,3.85867e+22,0.031002,"{'C': 1, 'epsilon': 0.6}"
SVR(),0.12311,3.82379e+07,0.077001,"{'C': 2, 'epsilon': 0.2, 'kernel': 'rbf'}"
RandomForestRegressor(),0.29521,3.42808e+07,0.039001,"{'max_depth': 11, 'min_samples_leaf': 1, 'min_..."
LGBMRegressor(),0.3103,3.39118e+07,0.016001,"{'boosting_type': 'gbdt', 'learning_rate': 0.1..."


In [144]:
recap_ghg_log

,R²,RMSE,Prediction time,Best parameters
Model for GHG log,,,,
LinearRegression(),-4.49413e+22,1.6217e+14,0.003005,{}
Ridge(),-3.21074e+23,4.33461e+14,0.002,{'alpha': 4.641589}
Lasso(),-6.76463e+23,6.29172e+14,0.001998,{'alpha': 0.002477}
SVR(kernel='linear'),-1.08898e+29,2.5244e+17,0.065007,"{'C': 1, 'epsilon': 0.4}"
SVR(),0.16586,698.661,0.069023,"{'C': 3, 'epsilon': 0.5, 'kernel': 'rbf'}"
RandomForestRegressor(),0.64996,452.594,0.020002,"{'max_depth': 11, 'min_samples_leaf': 2, 'min_..."
LGBMRegressor(),0.47625,553.618,0.016003,"{'boosting_type': 'dart', 'learning_rate': 0.1..."


In [41]:
recap_energy_log.iloc[-1,-1]

{'boosting_type': 'gbdt',
 'learning_rate': 0.1,
 'max_depth': 3,
 'n_estimators': 300}

In [130]:
recap_ghg_log.iloc[-2,-1]

{'max_depth': 11,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 100,
 'random_state': 0}